In [8]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [16]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [17]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [18]:
# Load the data
file_path = Path('./Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,19-Mar,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,19-Mar,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,19-Mar,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,19-Mar,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,19-Mar,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [19]:
# Create our features
#X = YOUR CODE HERE loan_status
X = df.drop(columns="loan_status")

# Create our target
#y = YOUR CODE HERE
y = df["loan_status"]

In [20]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [21]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [33]:
cat_cols = X.select_dtypes(exclude=['int', 'float']).columns.tolist()

In [34]:
dummy_df = pd.get_dummies(X[cat_cols])
dummy_df.head()

,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,issue_d_19-Feb,issue_d_19-Jan,issue_d_19-Mar,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_19-Apr,next_pymnt_d_19-May,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,1,1,0,1,1
1,0,1,0,0,0,0,1,0,0,1,1,0,1,0,1,1,0,1,1
2,0,1,0,0,0,0,1,0,0,1,1,0,1,0,1,1,0,1,1
3,0,0,0,1,0,0,1,0,0,1,1,0,1,0,1,1,0,1,1
4,0,1,0,0,1,0,0,0,0,1,1,0,1,0,1,1,0,1,1


In [35]:
X_new = pd.concat([X, dummy_df], axis=1)
X_new.shape

(68817, 104)

In [36]:
X_new.drop(cat_cols, axis=1, inplace=True)
X_new.shape

(68817, 95)

In [37]:
X = X_new

In [38]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
(X_train.shape, X_test.shape)

((51612, 95), (17205, 95))

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [39]:
#Train and Predict using the Balanced Random Forest Classifier
# Resample the training data with the RandomOversampler
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
#Training 
brf.fit(X_train, y_train) 
# Using features of  
y_pred = brf.predict(X_test)

In [41]:
# Calculated the balanced accuracy score
# comparing the actual output to the predicted
balanced_accuracy_score(y_test, y_pred)

0.7844011748069183

In [42]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   59,    28],
       [ 1872, 15246]], dtype=int64)

In [43]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.68      0.89      0.06      0.78      0.59        87
   low_risk       1.00      0.89      0.68      0.94      0.78      0.62     17118

avg / total       0.99      0.89      0.68      0.94      0.78      0.62     17205



In [54]:
column_names_features_importance[:10]

[{'name': 'loan_amnt', 'importance': 0.012440616346608431},
 {'name': 'int_rate', 'importance': 0.030267838474527374},
 {'name': 'installment', 'importance': 0.017152874888893842},
 {'name': 'annual_inc', 'importance': 0.01781406866335307},
 {'name': 'dti', 'importance': 0.01626758722734418},
 {'name': 'delinq_2yrs', 'importance': 0.003677884598601189},
 {'name': 'inq_last_6mths', 'importance': 0.0036191968322430487},
 {'name': 'open_acc', 'importance': 0.01261310712060957},
 {'name': 'pub_rec', 'importance': 0.0013743375521492434},
 {'name': 'revol_bal', 'importance': 0.016620756943505968}]

In [55]:
feature_importances[:5]

[0.012440616346608431,
 0.030267838474527374,
 0.017152874888893842,
 0.01781406866335307,
 0.01626758722734418]

In [57]:
column_names[:5]

['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti']

In [66]:
# List the features sorted in descending order by feature importance fewer steps
foo = list(zip(column_names, feature_importances))
foo[:3]
sorted_importance= sorted(foo, key=lambda x: x[1], reverse=True)[:5]
[x[0] for x in sorted_importance][:5]

['total_rec_prncp',
 'total_pymnt_inv',
 'total_rec_int',
 'last_pymnt_amnt',
 'total_pymnt']

In [68]:
# List the features sorted in descending order by feature importance
brf.feature_importances_
feature_importances = list(brf.feature_importances_)
column_names = X.columns.values.tolist()

column_names_features_importance =[]

for idx, col_name in enumerate(column_names):
    column_names_features_importance.append({"name": col_name, "importance": feature_importances[idx]})
sorted_cols = sorted(column_names_features_importance, key = lambda x: x['importance'], reverse=True)

#for obj in sorted_cols:
#    print(obj['name']  

[x['name'] for x in sorted_cols]

['total_rec_prncp',
 'total_pymnt_inv',
 'total_rec_int',
 'last_pymnt_amnt',
 'total_pymnt',
 'int_rate',
 'issue_d_19-Jan',
 'out_prncp',
 'annual_inc',
 'installment',
 'revol_bal',
 'dti',
 'il_util',
 'mo_sin_old_rev_tl_op',
 'avg_cur_bal',
 'total_rev_hi_lim',
 'out_prncp_inv',
 'tot_hi_cred_lim',
 'max_bal_bc',
 'total_bc_limit',
 'bc_util',
 'mo_sin_old_il_acct',
 'tot_cur_bal',
 'mths_since_recent_inq',
 'open_acc',
 'total_il_high_credit_limit',
 'all_util',
 'loan_amnt',
 'total_bal_ex_mort',
 'mths_since_rcnt_il',
 'bc_open_to_buy',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mths_since_recent_bc',
 'total_bal_il',
 'num_sats',
 'num_rev_accts',
 'issue_d_19-Mar',
 'total_acc',
 'num_il_tl',
 'num_actv_rev_tl',
 'pct_tl_nvr_dlq',
 'total_cu_tl',
 'inq_fi',
 'num_bc_tl',
 'inq_last_12m',
 'num_actv_bc_tl',
 'num_rev_tl_bal_gt_0',
 'total_rec_late_fee',
 'open_act_il',
 'open_il_24m',
 'num_tl_op_past_12m',
 'percent_bc_gt_75',
 'num_op_rev_tl',
 'open_rv_24m',
 'acc_open_

### Easy Ensemble AdaBoost Classifier

In [73]:
# Train the Classifier
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100, random_state=0)
# Training the classifier
clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100, random_state=0)

In [74]:
#making the prediction from the test data
y_predict = clf.predict(X_test)

In [75]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.7844011748069183

In [76]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   59,    28],
       [ 1872, 15246]], dtype=int64)

In [77]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.68      0.89      0.06      0.78      0.59        87
   low_risk       1.00      0.89      0.68      0.94      0.78      0.62     17118

avg / total       0.99      0.89      0.68      0.94      0.78      0.62     17205

